# David Fleming, Jan. 22 2016
This notebook copies the stellar and gaseous disk properties from a previous tipsy snapshot generated by ICgen (see diskpy) and inserts a planet of user-specified mass and semimajor axis along the x-axis.  The planet is assumed to feel the gravity of the binary and the gas particles with a cylindrical radius less than its own for its initial keplerian velocity computation.

In [1]:
%matplotlib inline
#Imports required to run

import pynbody
SimArray = pynbody.array.SimArray
from matplotlib.pylab import *
import matplotlib.pylab as plt
import pynbody.plot.sph as sph
import numpy as np
from scipy import interpolate

#Tell code where my diskpy distribution is
import os

#Import diskpy and my modules from ICgen submodule
import diskpy
from diskpy.ICgen import *
from diskplot import *

from __future__ import division

#Typical plot parameters that make for pretty plots
plt.rcParams['figure.figsize'] = (10,8)
from astroML.plotting import setup_text_plots
setup_text_plots(fontsize=20, usetex=True)

rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
mpl.rcParams['font.size'] = 20.0

# Load initial planet-less ICs, subtract off center of mass

In [3]:
#Change directory to where the data is
cwd = '/astro/users/dflemin3/Desktop/sims/k38_e_0.1032_final'
os.chdir(cwd)

In [4]:
#Load Snapshot 
s = pynbody.load('k38_snap.std')

#Center on barycenter
diskCom = binaryUtils.computeCOM(s.stars,s.gas,starFlag=True,gasFlag=True)
s['pos'] -= diskCom

# Make a new pynbody snapshot object with a dark matter (dm) particle to act as our planet.
The dark matter particle is nice because it won't interact with any SPH stuff as expected for a planet decoupled from the disk AND dark matter particles don't accrete like stars so we don't have to do anything funny with the stellar sink radius.

In [5]:
s2 = pynbody.new(star=2, gas=100000, dm=1)

# Copy all the properties from the old snapshot into the new one for both the gas and star keys

In [6]:
# Copy star properties
s2.star[0]['pos'] = s.star[0]['pos']
s2.star[0]['vel'] = s.star[0]['vel']
s2.star[1]['pos'] = s.star[1]['pos']
s2.star[1]['vel'] = s.star[1]['vel']
s2.star[0]['mass'] = s.star[0]['mass']
s2.star[1]['mass'] = s.star[1]['mass']
s2.star['metals'] = s.star['metals']
s2.star[0]['eps'] = s.star[0]['eps'] 
s2.star[1]['eps'] = s.star[1]['eps']

# Make sure stars are sinks
s2.star['tform'] = -1.0

#Assign gas particles with calculated/given values from above
s2.gas['pos'] = s.gas['pos']
s2.gas['vel'] = s.gas['vel']
s2.gas['temp'] = s.gas['temp']
s2.gas['rho'] = s.gas['rho']
s2.gas['eps'] = s.gas['eps']
s2.gas['mass'] = s.gas['mass']
s2.gas['metals'] = s.gas['metals']

# Generate a Binary object with the "planet" as the secondary to get its position, velocity subject to whatever orbital elements I want.  Allows me to arbitrarily change the orbit.

In [25]:
# X = [e, a, i, Omega, w, nu]
X = [0.0, 1.5, 0.0, 0.0, 0.0, 0.0]

# Find gas mass interior to orbit where a_planet = X[1]
gas_mask = s2.gas['rxy'].in_units('au') < X[1]
m_gas = np.sum(s.gas['mass'][gas_mask])

# m1 = mass of primary = mass of binary + mass of gas interior to a_planet
m1 = m_gas + np.sum(s.stars['mass'])

# m2 = mass of planet in solar masses
m2 = 3.63e-4 * s2.stars[0]['mass']

# Make planet Binary object, generate the positions, velocities
planet = binary.Binary(X,m1=m1,m2=m2,state='Kepler')
r1, r2, v1, v2 = planet.generateICs()

# Note: I'm going to assume, rather safely, that the planet does not change the binary's positions, velocities, etc. 
This assumption will suffice for m_planet < m_jup probs

In [35]:
r_unit = s2['pos'].units
v_unit = s2['vel'].units
m_unit = s2['mass'].units

s2.dm['pos'] = SimArray(r2,r_unit)
s2.dm['vel'] = SimArray(v2,v_unit)
s2.dm['mass'] = SimArray(m2,m_unit)

# For the planet smoothing radius, use fraction of the hill radius (consistent with minium smoothing length of Kley+2014)

$$
r_H = a_p*(m_p/3M_{bin})^{1/3}
$$

In [36]:
rh = np.power(planet.a*(m2/np.sum(s.stars['mass'])),1./3.)
eps = 0.3*rh

In [44]:
s2.dm['eps'] = eps

# Write to a new snapshot!

In [45]:
s2.write(fmt=pynbody.tipsy.TipsySnap, filename="k38_planet.std")

/astro/users/dflemin3/.local/lib/python2.7/site-packages/pynbody/simdict.py:92: RuntimeWarning: Assuming default value for property 'a'=1.00e+00
  name, value), RuntimeWarning)
